Code Used for the evaluation of the standing balance with perturbation tasks. 

In [ ]:
import gym
from gym import spaces
from tqdm import tqdm
import mujoco_py
import mj_envs
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import EvalCallback
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
from tensorboard.backend.event_processing.reservoir import Reservoir
from plottingStuff import storeData   
import matplotlib.pyplot as plt
import skvideo.io
import os
import pickle
import random
from matplotlib.backends.backend_pdf import PdfPages 
from plottingStuff import storeData, plotMuscleActivation, plotMuscleForce, plotMuscleLength, plotJointQpos, plotCOM, plotErrorAndReward

def storeData(env, model, steps, env_name, file_name):

    dataStore = {}
    dataStore['modelInfo'] = {}
    dataStore['modelInfo']['bodyNames'] = env.sim.model.body_names
    dataStore['modelInfo']['jointNames'] = env.sim.model.joint_names
    dataStore['modelInfo']['tendonNames'] = env.sim.model.tendon_names
    dataStore['modelInfo']['rewardWeights'] = env.rwd_keys_wt
    dataStore['modelInfo']['trainingSteps'] = 20000000
    dataStore['modelInfo']['testSteps'] = steps
    dataStore['modelInfo']['perturbationMagnitude'] = env.perturbation_magnitude
    dataStore['modelInfo']['perturbationTime'] = env.perturbation_time
    for _ in tqdm(range(dataStore['modelInfo']['testSteps'])):       
        obs = env.get_obs_vec()
        action, __ = model.predict(obs, deterministic=True)
        
        env.sim.data.ctrl[:] = action
        env.step(action)
        
        if _ == 0 :                
            dataStore['modelInfo']['targetPosition'] = {}
            dataStore['modelInfo']['reachError'] = {}
            dataStore['modelInfo']['tipPosition'] = {}
            dataStore['modelInfo']['rewardDict'] = {}
            
            dataStore['stateInfo'] = {}

            dataStore['jointInfo'] = {}
            dataStore['jointInfo']['qpos'] = {}
            dataStore['jointInfo']['qvel'] = {}
            dataStore['jointInfo']['qacc'] = {}
            dataStore['jointInfo']['torque'] = {}

            dataStore['jointInfo']['ROM'] = {}

            data_joint = {}
            for i, joint in enumerate(env.sim.model.joint_names):
                    data_joint[joint] = {}
                    data_joint[joint]['MinRom'] =  env.sim.model.jnt_range[env.sim.model.joint_name2id(joint), 0].copy()
                    data_joint[joint]['MaxRom'] = env.sim.model.jnt_range[env.sim.model.joint_name2id(joint), 1].copy()
            dataStore['jointInfo']['ROM'] =  data_joint
            
            dataStore['bodyInfo'] = {}
            dataStore['bodyInfo']['com'] = {}   
            dataStore['bodyInfo']['bos'] = {}
            dataStore['bodyInfo']['xipos'] = {}
            dataStore['bodyInfo']['xpos'] = {} 
            dataStore['bodyInfo']['grf'] = {}
            dataStore['bodyInfo']['grf']['rToes'] = {}
            dataStore['bodyInfo']['grf']['lToes'] = {}
            dataStore['bodyInfo']['grf']['rCal'] = {}
            dataStore['bodyInfo']['grf']['lCal'] = {}
                
            dataStore['muscleInfo'] = {}
            dataStore['muscleInfo']['action'] = {}
            dataStore['muscleInfo']['muscleForce'] = {}
            dataStore['muscleInfo']['muscleActivation'] = {}
            dataStore['muscleInfo']['muscleLength'] = {}
            dataStore['muscleInfo']['muscleMoment'] = {}
            dataStore['muscleInfo']['muscleVelocity'] = {}

            dataStore['tensorBoard'] = {}
            dataStore['tensorBoard']['meanReward'] = {}
            dataStore['tensorBoard']['epMeanReward'] = {}
            
            dataStore['videoInfo'] = {}
            dataStore['videoInfo']['framesSide'] = {}
            dataStore['videoInfo']['framesFront'] = {}
            
            state = []
            targetPosition, reachError, tipPosition, rewardDict = [], [], [], []
            qpos, qvel, qacc, torque = [], [], [], []
            com, bos, xpos, xipos, grf_rToes, grf_lToes, grf_rCal, grf_lCal = [], [], [], [], [], [], [], []
            muscleAction, muscleForce, muscleActivation, muscleLength, muscleMoment, muscleVelocity = [], [], [], [], [], []
            frames_side, frames_front = [], []

        # state.append(env.env.get_env_state())    
        targetPosition.append(env.obs_dict['target_pos'].copy())
        reachError.append(env.obs_dict['reach_err'].copy())
        tipPosition.append(env.obs_dict['tip_pos'].copy())
        rewardDict.append(env.get_reward_dict(env.get_obs_dict(env.sim)).copy())

        # Joint Info
        qpos.append(env.sim.data.qpos.copy())
        qvel.append(env.sim.data.qvel.copy())
        qacc.append(env.sim.data.qacc.copy())
        torque.append(env.sim.data.qfrc_actuator[6:].copy())
        
        # Body Info
        com.append(env.obs_dict['com'].copy())
        bos.append(env.obs_dict['base_support'].copy())
        xpos.append(env.sim.data.body_xpos.copy())
        xipos.append(env.sim.data.xipos.copy())
        grf_rToes.append(env.sim.data.get_sensor('r_toes').copy())
        grf_lToes.append(env.sim.data.get_sensor('l_toes').copy())
        grf_rCal.append(env.sim.data.get_sensor('r_foot').copy())
        grf_lCal.append(env.sim.data.get_sensor('l_foot').copy())
        
        # Muscle Info
        muscleAction.append(action.copy())
        muscleForce.append(env.sim.data.actuator_force.copy())
        muscleActivation.append(env.sim.data.act.copy())       
        muscleLength.append(env.sim.data.actuator_length.copy())
        muscleMoment.append(env.sim.data.actuator_moment.copy())
        muscleVelocity.append(env.sim.data.actuator_velocity.copy())
        
        geom_1_indices = np.where(env.sim.model.geom_group == 1)
        env.sim.model.geom_rgba[geom_1_indices, 3] = 0
        frame_side = env.sim.sim.render(width=800, height=800,mode='offscreen', camera_name='side_view') # if slow see https://github.com/facebookresearch/myosuite/blob/main/setup/README.md
        frames_side.append(frame_side[::-1,:,:])
        frame_front = env.sim.sim.render(width=800, height=800,mode='offscreen', camera_name='front_view') # if slow see https://github.com/facebookresearch/myosuite/blob/main/setup/README.md
        frames_front.append(frame_front[::-1,:,:])


    tb_logdir = "/home/carlota/Documents/MyoLeg_script_test/standingBalance/temp_env_tensorboard/"+ env_name + "_" + file_name + "_1"
    event_accumulator = EventAccumulator(tb_logdir)
    event_accumulator.Reload()
    events = event_accumulator.Scalars('rollout/ep_rew_mean')
    ep_rew_mean = [x.value for x in events]
    events = event_accumulator.Scalars('eval/mean_reward')
    mean_reward= [x.value for x in events]

    dataStore['modelInfo']['targetPosition'] = targetPosition
    dataStore['modelInfo']['rewardDict'] = rewardDict
    dataStore['modelInfo']['tipPosition'] = tipPosition
    dataStore['modelInfo']['reachError'] = reachError

    #dataStore['stateInfo'] = state

    dataStore['jointInfo']['qpos'] = qpos
    dataStore['jointInfo']['qvel'] = qvel
    dataStore['jointInfo']['qacc'] = qacc
    dataStore['jointInfo']['torque'] = torque

    dataStore['bodyInfo']['com'] = com
    dataStore['bodyInfo']['bos'] = bos
    dataStore['bodyInfo']['xpos'] = xpos
    dataStore['bodyInfo']['xipos'] = xipos
    dataStore['bodyInfo']['grf']['rToes'] = grf_rToes
    dataStore['bodyInfo']['grf']['lToes'] = grf_lToes
    dataStore['bodyInfo']['grf']['rCal'] = grf_rCal
    dataStore['bodyInfo']['grf']['lCal'] = grf_lCal

    dataStore['muscleInfo']['action'] = muscleAction
    dataStore['muscleInfo']['muscleForce'] = muscleForce
    dataStore['muscleInfo']['muscleActivation'] = muscleActivation      
    dataStore['muscleInfo']['muscleLength'] = muscleLength
    dataStore['muscleInfo']['muscleMoment'] = muscleMoment
    dataStore['muscleInfo']['muscleVelocity'] = muscleVelocity

    dataStore['tensorBoard']['epMeanReward'] = ep_rew_mean    
    dataStore['tensorBoard']['meanReward'] = mean_reward

    dataStore['videoInfo']['framesSide'] = frames_side
    dataStore['videoInfo']['framesFront'] = frames_front

    return dataStore

In [2]:
# Load policy
nb_seed = 1

env_name = 'myoLegReachFixed-v2'
dir_path = "/home/carlota/Documents/MyoLeg_script_test/standingBalance/policy_best_model/" + env_name
policy_name = '2023_05_25_17_25_29'
policy_path = dir_path + '/' + policy_name + '/best_model.zip'

model = PPO.load(policy_path)
env = gym.make(env_name) 
env.reset()
random.seed()

episodes = 200 # number of episodes for evaluation
steps = 1000 # steps in each episode
print(f'{env_name} \n {env.rwd_keys_wt} \n {env.horizon}')



/home/carlota/miniconda3/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


RoboHive:> For environment credits, please cite -
    RoboHive: A unified framework for robot learning | https://sites.google.com/view/robohive
    
Configuring a new session for default_robot(sim)
myoLegReachFixed-v2 
 {'positionError': 1, 'smallErrorBonus': 0, 'metabolicCost': 1, 'highError': 0, 'centerOfMass': 1} 
 200


/home/carlota/miniconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [3]:
pkl_path = '/home/carlota/Documents/output/PKL/' + env_name + '/' + policy_name + '/'
os.makedirs(pkl_path, exist_ok=True)
video_path = '/home/carlota/Documents/output/videos/' + env_name + '/' + policy_name + '/'
os.makedirs( video_path, exist_ok=True)
    
for ep in range(episodes):
    print(f'### EPISODE {ep} ###')
    env = gym.make(env_name) 
    env.reset()
    random.seed()
    data = {}
    data = storeData(env, model, steps, env_name, policy_name)
    frames_front = data['videoInfo']['framesFront']
    frames_side = data['videoInfo']['framesSide']
    data['videoInfo'] = {}

    with open(pkl_path + policy_name+ '_' + str(ep) + '.pkl', 'wb') as fp:
        pickle.dump(data, fp)
        print('dictionary saved successfully to file')
    
    skvideo.io.vwrite(video_path + policy_name+ '_' + str(ep) + '_side' + '.mp4', np.asarray(frames_side), inputdict = {'-r':'100'}, outputdict={"-pix_fmt": "yuv420p"})
    skvideo.io.vwrite(video_path + policy_name+ '_' + str(ep) +  '_front' + '.mp4', np.asarray(frames_front), inputdict = {'-r':'100'}, outputdict={"-pix_fmt": "yuv420p"})
    print('Movie saved successfully to file')
    

### EPISODE 0 ###
RoboHive:> For environment credits, please cite -
    RoboHive: A unified framework for robot learning | https://sites.google.com/view/robohive
    
Configuring a new session for default_robot(sim)
Closing default_robot(sim)


100%|██████████| 1000/1000 [11:32<00:00,  1.44it/s]


dictionary saved successfully to file
Movie saved successfully to file
### EPISODE 1 ###
RoboHive:> For environment credits, please cite -
    RoboHive: A unified framework for robot learning | https://sites.google.com/view/robohive
    
Configuring a new session for default_robot(sim)
Closing default_robot(sim)


100%|██████████| 1000/1000 [11:34<00:00,  1.44it/s]


dictionary saved successfully to file
Movie saved successfully to file
### EPISODE 2 ###
RoboHive:> For environment credits, please cite -
    RoboHive: A unified framework for robot learning | https://sites.google.com/view/robohive
    
Configuring a new session for default_robot(sim)
Closing default_robot(sim)


...

dictionary saved successfully to file
Movie saved successfully to file
### EPISODE 18 ###
RoboHive:> For environment credits, please cite -
    RoboHive: A unified framework for robot learning | https://sites.google.com/view/robohive
    
Configuring a new session for default_robot(sim)
Closing default_robot(sim)


100%|██████████| 1000/1000 [11:37<00:00,  1.43it/s]


dictionary saved successfully to file
Movie saved successfully to file
### EPISODE 19 ###
RoboHive:> For environment credits, please cite -
    RoboHive: A unified framework for robot learning | https://sites.google.com/view/robohive
    
Configuring a new session for default_robot(sim)
Closing default_robot(sim)


100%|██████████| 1000/1000 [11:53<00:00,  1.40it/s]


dictionary saved successfully to file
Movie saved successfully to file
